In [ ]:
%matplotlib widget
import os,sys
import string
from pathlib import Path

import numpy as np
import pandas as pd
from pyproj import Proj, transform

# avoid tkinter issue
import matplotlib
matplotlib.use('tkagg')

# bokeh plotting
from bokeh.io import output_notebook
from bokeh.plotting import figure,show
from bokeh.models.tools import HoverTool, TapTool
from bokeh.models import CustomJS, ColumnDataSource, LabelSet
from bokeh.tile_providers import get_provider, Vendors

# pyviz
import param
import panel as pn
import holoviews as hv
from holoviews import opts, streams
from holoviews.plotting.links import DataLink
import hvplot.pandas

#output_notebook()
hv.notebook_extension('bokeh')
pn.extension('vtk')

# import DoZen
# add parent directory to path
try:
    # works when called externally via panel serve
    module_path = os.path.abspath(os.path.join(__file__,'..','..'))
except NameError:
    # works in notebook
    module_path = str(Path().resolve().parent)
if module_path not in sys.path:
    sys.path.append(module_path)
from dozen import z3d_directory, z3dio, util

In [ ]:
# Load campaign-specific variables from csv
settings = pd.read_csv('timeline_settings.csv',index_col=None)

In [ ]:
# Get campaign from user
choosing = True
while(choosing):
    print(settings.name)
    t = input('Enter campaign index number: ')
    try:
        campaign_index = int(t)
        campaign_name = settings.loc[campaign_index,'name']
        print(campaign_name)
        choosing = False
    except ValueError:
        print('Index must be a number')
    except KeyError:
        print('Index '+str(campaign_index)+' not found')
        print('\n')

In [ ]:
# Set campaign-specific variables
campaign = settings.loc[campaign_index]
rx_dir = campaign.rx_dir
tx_dir = campaign.tx_dir
location_file = campaign.rx_station_location_file
min_end = pd.Timestamp(campaign.min_end,tz='US/Mountain')
max_start = pd.Timestamp(campaign.max_start,tz='US/Mountain')
min_easting = campaign.min_easting
max_easting = campaign.max_easting
min_northing = campaign.min_northing
max_northing = campaign.max_northing
utm_zone = campaign.utm_zone

In [ ]:
# Read z3d directory info
rx_full = z3d_directory.get_z3d_directory_info(initialdir=rx_dir,ask_dir=False)
tx_full = z3d_directory.get_z3d_directory_info(initialdir=tx_dir,ask_dir=False)

In [ ]:
# ID duplicates
rx_full['original'] = ~rx_full.duplicated(subset='filename',keep='first')
tx_full['original'] = ~tx_full.duplicated(subset='filename',keep='first')

In [ ]:
# How many z3d files are there?
print('{} rx files'.format(len(rx_full)))
# Count duplicated files
print('{} duplicated rx files'.format((~rx_full.original).sum()))
# Count invalid files
print('{} invalid rx files'.format((~rx_full.valid).sum()))

# Do the same for tx
print('{} tx files'.format(len(tx_full)))
print('{} duplicated tx files'.format((~tx_full.original).sum()))
print('{} invalid tx files'.format((~tx_full.valid).sum()))

In [ ]:
# remove duplicates, keeping only the first
rx = rx_full.drop_duplicates(subset='filename',keep='first',inplace=False).copy()
tx = tx_full.drop_duplicates(subset='filename',keep='first',inplace=False).copy()
# drop invalid files
rx.drop(rx[~rx.valid].index,inplace=True)
tx.drop(tx[~tx.valid].index,inplace=True)

In [ ]:
#tx.loc[tx.iloc[[37,38]].index]
tx_full.loc[tx.iloc[[37,38]].index]

In [ ]:
# ID z3ds outside of date range
def after_min_end(x):
    try:
        return x>min_end
    except TypeError:
        return False

def before_max_start(x):
    try:
        return x<max_start
    except TypeError:
        return False

rx_full['in_range'] = rx_full.end.apply(after_min_end) & rx_full.start.apply(before_max_start)
tx_full['in_range'] = tx_full.end.apply(after_min_end) & tx_full.start.apply(before_max_start)

In [ ]:
# Are there old z3ds in this folder?
print('{} rx files end before {}'.format((rx.end<min_end).sum(),min_end))
print('{} rx files start after {}'.format((rx.start>max_start).sum(),max_start))

# Do the same for tx
print('{} tx files end before {}'.format((tx.end<min_end).sum(),min_end))
print('{} tx files start after {}'.format((tx.start>max_start).sum(),max_start))

In [ ]:
# drop files outside of specified date range
rx.drop(rx[rx.end<min_end].index,inplace=True)
tx.drop(tx[tx.end<min_end].index,inplace=True)

In [ ]:
# now, what's left?
print('{} rx files'.format(len(rx)))
# Count duplicated files
print('{} duplicated rx files'.format(rx.duplicated(subset='filename',keep='first').sum()))
# Count invalid files
print('{} invalid rx files'.format((~rx.valid).sum()))
# Are there old z3ds in this folder?
print('{} rx files end before {}'.format((rx.end<min_end).sum(),min_end))
print('{} rx files start after {}'.format((rx.start>max_start).sum(),max_start))

# Do the same for tx
print('{} tx files'.format(len(tx)))
print('{} duplicated tx files'.format(tx.duplicated(subset='filename',keep='first').sum()))
print('{} invalid tx files'.format((~tx.valid).sum()))
print('{} tx files end before {}'.format((tx.end<min_end).sum(),min_end))
print('{} tx files start after {}'.format((tx.start>max_start).sum(),max_start))

In [ ]:
# remind me, what are the fields in rx?
rx.columns

In [ ]:
# To get job number, station name, and run robustly, change this regex to suit your naming system
parsed_job_number = pd.to_numeric(rx.job_number.str.replace("[^\d-]",""),downcast='integer')
parsed_station_folder = pd.to_numeric(rx.filepath.str.extract(r'/(?:BC|sub|Sub|SUB|LMA)(\d+)[a-zA-Z]+/')[0])
parsed_run = rx.job_number.astype(str).str[-1].fillna('')
parsed_run = parsed_run.str.replace('\d+','')
parsed_station_name = rx.rx_station.astype(int).astype(str) + parsed_run

In [ ]:
rx['parsed_run'] = parsed_run
rx['parsed_station_name'] = parsed_station_name
# compute web mercator coordinates
utm_proj = Proj(proj="utm",zone=utm_zone,ellps='WGS84')
web_mercator = Proj(init='epsg:3857')
rx['x_web_mercator'],rx['y_web_mercator'] = transform(utm_proj,web_mercator,rx.easting.values,rx.northing.values)

In [ ]:
# prepare full dataframes with dummy values
rx_full['rx_station_qc'] = 0
rx_full['run_qc'] = ''
rx_full['easting_utm_qc'] = 0.0
rx_full['northing_utm_qc'] = 0.0
tx_full['rx_station_qc'] = 0

In [ ]:
# Plot rx_station column using matplotlib
#ax = rx.plot('easting','northing',kind='scatter')
#for i,v in rx.iterrows():
    #ax.text(v['easting'],v['northing'],v['rx_station'])

In [ ]:
def label_hook(plot,element):
    '''
    Hook to use Bokeh for labels
    Holoview's labels don't position correctly
    '''
    rcsource = ColumnDataSource(rx)
    station_labels = LabelSet(x='x_web_mercator',y='y_web_mercator',text='parsed_station_name',source=rcsource)
    plot.handles['plot'].add_layout(station_labels)

In [ ]:
rx_locations = rx.hvplot.scatter(x='x_web_mercator',
                                 y='y_web_mercator',
                                 padding=0.2,
                                 selection_color='red'
                                )
sel = streams.Selection1D(source=rx_locations)

dtools = ['save','pan','box_zoom','reset']
stools = ['lasso_select']
base_map = hv.element.tiles.StamenTerrain()
hvp = (base_map*rx_locations).opts(
#hvp = (rx_locations).opts(
    opts.Scatter(default_tools=dtools,
                 tools=stools,
                 active_tools=stools,
                 hooks=[label_hook],
                 width=600,
                 height=400),
    opts.Tiles(default_tools=dtools),
    opts.Table(editable=True))

In [ ]:
def averaged_label_hook(plot,element):
    '''
    Hook to use Bokeh for labels
    Holoview's labels don't position correctly
    '''
    
    #parsed_run = rx.job_number.astype(str).str[-1]
    #parsed_run = parsed_run.str.replace('\d+','')
    #parsed_station_name = rx.rx_station.astype(int).astype(str) + parsed_run
    
    station_name = rxla.averaged_points.rx_station
    run = rxla.averaged_points.run.astype(str).str.replace('Do not change','')
    rxla.averaged_points['station_name'] = station_name + run
    rcsource = ColumnDataSource(rxla.averaged_points)
    station_labels = LabelSet(x='x_web_mercator',y='y_web_mercator',text='station_name',source=rcsource,text_color='green')
    plot.handles['plot'].add_layout(station_labels)

In [ ]:
class RX_Location_Averager(param.Parameterized):
    '''
    Let's do this
    '''
    station_number = param.Integer(default=0)
    station_run = param.Selector(default='Do not change',objects=['Do not change','a','b','c','d','e','f','g'])
    
    #guess_station_number = param.Action(lambda x: x.guess_station_number_from_selected(), label='Guess station number')
    add_to_list = param.Action(lambda x: x.add_selected_to_list(), label='Add to list')
    remove_from_list = param.Action(lambda x: x.remove_selected_from_list(), label='Remove from list')
    # hidden parameter for updating plot and table
    _update_view = param.Action(lambda x: x.param.trigger('_update_view'), 
                                label='Update view', 
                                precedence=-1)
    
    averaged_points = pd.DataFrame({'x_web_mercator':[],
                                    'y_web_mercator':[],
                                    'rx_station':[],
                                    'run':[],
                                    'station_name':[]})
    
    def add_selected_to_list(self):
        '''
        triggered when "Add to list" button is clicked
        Computes an average location from selected points
        '''
        x_avg = rx['x_web_mercator'].iloc[sel.index].mean()
        y_avg = rx['y_web_mercator'].iloc[sel.index].mean()
        this_rx_station = str(self.station_number)
        this_run = self.station_run
        new_location = pd.DataFrame({'x_web_mercator':[x_avg],
                                     'y_web_mercator':[y_avg],
                                     'rx_station':[this_rx_station],
                                     'run':[this_run],
                                     'station_name':[this_rx_station+this_run],
                                     'full_indeces':[rx.iloc[sel.index].index]})
        if not (np.isnan(x_avg) or np.isnan(y_avg)):
            self.averaged_points = self.averaged_points.append(new_location,ignore_index=True,sort=False)
        self.param.trigger('_update_view')
    
    def remove_selected_from_list(self):
        '''
        triggered when "Remove from list" button is clicked
        removes selected location from table
        '''
        idx = self.averaged_points.index[self.table_selection.index]
        new_df = self.averaged_points.drop(self.averaged_points.index[self.table_selection.index],inplace=False)
        self.averaged_points = new_df
        self.param.trigger('_update_view')
        
    
    @param.depends('_update_view')
    def averaged_points_plot(self):
        '''
        plot averaged points in green,
        overlayed on all points (hvp)
        '''
        avp = self.averaged_points.hvplot.scatter(
            x='x_web_mercator',
            y='y_web_mercator',
            padding=0.2,
            color='green',
            tools=[]
        ).opts(
            opts.Scatter(default_tools=dtools,
                         tools=[],
                         hooks=[averaged_label_hook])
        )
        return (hvp*avp).opts(opts.Tiles(default_tools=dtools),
                              opts.Scatter(default_tools=dtools))
    
    @param.depends('_update_view')
    def averaged_points_table(self):
        '''
        table of averaged points
        '''
        table = hv.Table(self.averaged_points,
                         ['x_web_mercator','y_web_mercator'],
                         ['rx_station','run']).opts(
        opts.Table(editable=True,height=600)
        )
        self.table_selection = streams.Selection1D(source=table)
        return table
    
    def guess_station_number_from_selected(self,index=0):
        '''
        triggered when "Guess station number" button is clicked,
        or when lasso selection changes
        Populates station number from selected points
        '''
        try:
            station_number = rx['rx_station'].iloc[sel.index].mode()[0]
            if type(station_number) != int:
                station_number = int(station_number)
            self.station_number = station_number
        except:
            station_number = 0
        try:
            job_letter = rx['job_number'].iloc[sel.index].mode()[0]
            run = job_letter[-1]
            #self.station_run = run
        except:
            run = self.param.station_run.objects[-1]
            
    #@param.depends('save_csv')
    def save_locations_as_csv(self,e):
        filename = util.savefile(title='Save locations file',initialdir=str(Path.cwd()))
        avp = self.averaged_points
        # compute utm coordinates
        avp['x_utm'],avp['y_utm'] = transform(web_mercator,utm_proj,avp.x_web_mercator.values,avp.y_web_mercator.values)
        avp.to_csv(filename)
    
    def save_full_as_csv(self,e):
        '''
        save rx_full and tx_full dataframes as csv files
        '''
        filename = util.savefile(title='Save z3d metadata files',initialdir=str(Path.cwd()))
        if filename:
            # save station number and run to full dataframes for posterity
            rx_full_write = rx_full.copy()
            avp = self.averaged_points
            # compute utm coordinates
            avp['x_utm'],avp['y_utm'] = transform(web_mercator,
                                                  utm_proj,
                                                  avp.x_web_mercator.values,
                                                  avp.y_web_mercator.values)
            
            # apply properties to all pertinent rows
            for avp_row in avp.itertuples(index=False):
                full_indeces = avp_row.full_indeces
                rx_full_write.loc[full_indeces,'rx_station_qc'] = int(avp_row.rx_station)
                if avp_row.run == 'Do not change':
                    rx_full_write.loc[full_indeces,'run_qc'] = rx.loc[full_indeces,'parsed_run']
                else:
                    rx_full_write.loc[full_indeces,'run_qc'] = avp_row.run
                rx_full_write.loc[full_indeces,'easting_utm_qc'] = avp_row.x_utm
                rx_full_write.loc[full_indeces,'northing_utm_qc'] = avp_row.y_utm
            rx_full_write.to_csv(filename+'_rx.csv')
            tx_full.to_csv(filename+'_tx.csv')
        
    def panel(self):
        return pn.Row(pn.Column(self.param,self.averaged_points_plot),self.averaged_points_table)


In [ ]:
rxla = RX_Location_Averager()
sel.add_subscriber(rxla.guess_station_number_from_selected)
save_locations = pn.widgets.Button(name='Save locations')
save_full = pn.widgets.Button(name='Save z3d metadata')
save_locations.on_click(rxla.save_locations_as_csv)
save_full.on_click(rxla.save_full_as_csv)
pn.Column(rxla.panel(),pn.Row(save_locations,save_full)).servable()